In [1]:
import os.path
import pandas as pd
import numpy as np
import pymannkendall as mk
pd.options.mode.chained_assignment = None  # default='warn'

work_dir = os.path.join(globals()['_dh'][0], "data")

SHIFT = 2  # to start hydrological year from November (more useful for Lammin Suo)

In [2]:
def prepare(f, v=False):
    df = pd.read_csv(f, comment='#')
    try:
        datetimes = pd.to_datetime(df["DATE_TIME"])
    except KeyError:
        df.rename(columns={"START_DATE": "DATE_TIME"}, inplace=True)
        datetimes = pd.to_datetime(df["DATE_TIME"])
    df["DATE_TIME"] = datetimes
    df.set_index("DATE_TIME", inplace=True)
    if v:
        df.info()
    return df

In [8]:
def test_my_data(series):
    out = mk.original_test(series)
    print("%s trend" % out.trend)
    print("p-value: %.2f" % out.p)
    print("Theil-Sen estimator/slope: %.3f" % out.slope)

### Air temperature:

In [19]:
f = "meteo_daily_1952-2020_WS1.csv"

tmp = prepare(os.path.join(work_dir, f))

df = tmp.loc[(tmp.index.year >= 1950) & (tmp.index.year <= 2021)]

df = df.resample("M").agg({"MEAN_AIR_TEMP": np.mean})
df = df.shift()  # so the calendar year will start with the previous Dec, needed for resampling by Q
df = df.resample("Q").agg({"MEAN_AIR_TEMP": np.mean})  # 1st Q is the winter, 3rd Q is the summer

summer = df.loc[df.index.month == 9]
summer["HYDRO_YEAR"] = (summer.index.year - 1).astype(str) + "/" + summer.index.year.astype(str)
# display(summer)

test_my_data(summer["MEAN_AIR_TEMP"])

winter = df.loc[df.index.month == 3]
winter["HYDRO_YEAR"] = (winter.index.year - 1).astype(str) + "/" + winter.index.year.astype(str)
winter = winter.loc[winter.index.year > 1952]  # since first hydro year is truncated: Dec 1951 was not observed
winter = winter.loc[winter.index.year >= 1976]
# display(winter)

test_my_data(winter["MEAN_AIR_TEMP"])

increasing trend
p-value: 0.01
Theil-Sen estimator/slope: 0.019
increasing trend
p-value: 0.00
Theil-Sen estimator/slope: 0.104


### Precipitation:

In [16]:
f = "precipitation_daily_1952-2020.csv"

tmp = prepare(os.path.join(work_dir, f))

df = tmp.loc[tmp["GAUGE_NO"] == "P1"]

df["P_TOTAL"] = df["PRECIPITATION"]
df["P_LIQUID"] = np.where(df["SOLID"] == 0, df["PRECIPITATION"], np.nan)
df["P_SOLID"] = np.where(df["SOLID"] == 1, df["PRECIPITATION"], np.nan)
df = df.resample('M').sum()
# print(df)

# Shifting all the columns to obtain hydrological years instead of calendar:
df["P_TOTAL"] = df["P_TOTAL"].shift(SHIFT)
df["P_LIQUID"] = df["P_LIQUID"].shift(SHIFT)
df["P_SOLID"] = df["P_SOLID"].shift(SHIFT)
# print(df)

df = df.resample('Y').sum()
df["HYDRO_YEAR"] = (df.index.year - 1).astype(str) + "/" + df.index.year.astype(str)
df = df.iloc[1:]  # since first hydro year is truncated (there is no Oct-Dec of 1951 in dataset)

df.loc[df["P_LIQUID"] == 0, "P_LIQUID"] = np.nan
df.loc[df["P_SOLID"] == 0, "P_SOLID"] = np.nan

df["SP"] = df["P_SOLID"] / df["P_TOTAL"]  # S/P ratio
df.loc[df["SP"] == 0, "SP"] = np.nan

# precipitation form is being recorded since Jan 1964
# therefore the hydro year 1963/1964 is not "complete" and should be ommitted:
df.loc[df["HYDRO_YEAR"] == "1963/1964", "P_SOLID"] = np.nan
df.loc[df["HYDRO_YEAR"] == "1963/1964", "P_LIQUID"] = np.nan
df.loc[df["HYDRO_YEAR"] == "1963/1964", "SP"] = np.nan

# display(df)

test_my_data(df["PRECIPITATION"])

test_my_data(df["SP"]*100)  # k = 100 to obtai percents from 0..1 range

increasing trend
p-value: 0.00
Theil-Sen estimator/slope: 3.638
decreasing trend
p-value: 0.03
Theil-Sen estimator/slope: -0.126


### Peat temperature:

In [23]:
f = "peat-temp_hourly_1976-2020.csv"

tmp = prepare(os.path.join(work_dir, f))

tmp = tmp.loc[tmp["QC"] != 0]
tmp = tmp.loc[tmp.index.hour == 15]
tmp = tmp.loc[tmp["SITE"] == "PT1"]

# PEAT SURFACE
df = tmp.loc[tmp["DEPTH"] == 0]
# print(df)
df = df.resample("A")["PEAT_TEMP"].agg(["mean", "max"])
# display(df)

test_my_data(df["mean"])  # peat surface

df = tmp.loc[tmp["DEPTH"] == 320]
df = df.resample("A")["PEAT_TEMP"].agg(["mean"])

test_my_data(df["mean"])  # max available depth of 320 cm

increasing trend
p-value: 0.00
Theil-Sen estimator/slope: 0.062
increasing trend
p-value: 0.00
Theil-Sen estimator/slope: 0.024


### Groundwater temperature:

In [28]:
f = "groundwater-temp_daily_1974-2020.csv"

tmp = prepare(os.path.join(work_dir, f))
tmp.sort_index(inplace=True)

wells = sorted(tmp["GW_WELL"].unique())
# print("All the gw-wells in the dataset:", wells)

longest_series = []
for w in wells:
    df = tmp.loc[tmp["GW_WELL"] == w]
    obs_num = len(df.index)
    if obs_num >= 1661:
        longest_series.append(w)
print(longest_series)

df = tmp.loc[tmp["GW_WELL"].isin(longest_series)]
df = df.resample("AS-NOV")["GW_TEMP"].agg(["mean", "max", "min", "std"])
df["HYDRO_YEAR"] = (df.index.year - 1).astype(str) + "/" + df.index.year.astype(str)
# display(df)

test_my_data(df["mean"])

['G359', 'G365', 'G366', 'G367', 'G375', 'G376', 'G377', 'G379']
increasing trend
p-value: 0.00
Theil-Sen estimator/slope: 0.027


### Discharges:

In [31]:
f = "water-discharge_daily_1950-2020.csv"

# watershed areas are: 1.18, 0.32, 0.37, 0.10 sq. km for S, W1+W2, N, E
basin_area = 1.18 + 0.32 + 0.37 + 0.10

df = prepare(os.path.join(work_dir, f))

df = df.loc[df["SITE"].isin(("Western-1", "Western-2", "Northern", "Southern"))]
df = df.loc[df.index.year >= 1955]
df["VOLUME"] = df["DISCHARGE"] * 86400 / 1000000  # cubic cm -> cubic m

df = df.resample("AS-NOV").agg("sum")
df["HYDRO_YEAR"] = df.index.year.astype(str) + "/" + (df.index.year + 1).astype(str)
df = df.loc[df.index.year < 2020]
# display(df)

test_my_data(df["VOLUME"] / 1000 / basin_area)

no trend trend
p-value: 0.06
Theil-Sen estimator/slope: 1.561
